# Generative Artificial Intelligence using Langchain

---

## 📑 Contents

1. Making API Instance
2. MySQL Connection
3. Querying to MySQL
4. Few Shot Learning
5. Making OpenAI Embeddings
6. Querying thorugh Few Shot Learning

#  Project - Q/A Tool



## 1. Making model API Instance


In [10]:
from langchain_openai import OpenAI
from dotenv import load_dotenv

load_dotenv() # To use env variable (our API key)

# Test call to the model API

llm = OpenAI(model='gpt-3.5-turbo-instruct')

result = llm.invoke('What is the capital of South Korea')

print(result)



The capital of South Korea is Seoul.


## 2. Connecting to MySQL Database

In [13]:
from langchain.utilities import SQLDatabase

db_user = 'root'
db_password = 'root'
db_host = 'localhost'
db_name = 'atliq_tshirts'

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount

*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Blue	XS	43	2

## 3. Querying to MySQL

In [33]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT COUNT(*) AS total FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White'
SQLResult: [(1,)]
Answer:1 t-shirt
> Finished chain.


It is wrong answer.

In [34]:
qns1 = db_chain.run("SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White'
SQLQuery:SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'
SQLResult: [(40,)]
Answer:There are currently 40 Nike XS White t-shirts in stock.
> Finished chain.


In [35]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) AS inventory_price FROM t_shirts WHERE size = 'S';
SQLResult: [(Decimal('19133'),)]
Answer:The price of the inventory for all small size t-shirts is $19,133.
> Finished chain.


This is right answer.

In [36]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM((t.price - (t.price * (d.pct_discount / 100))) * t.stock_quantity) as revenue 
           FROM t_shirts t 
           INNER JOIN discounts d ON t.t_shirt_id = d.t_shirt_id 
           WHERE t.brand = 'Levi' AND CURDATE() = DATE(d.discount_id)
SQLResult: [(None,)]
Answer:Final answer: The store will generate $0 in revenue after discounts are applied for all Levi's T-shirts today.
> Finished chain.


This is wrong Answer.

In [43]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue 
FROM 
	(SELECT sum(price*stock_quantity) as total_amount, t_shirt_id 
	FROM t_shirts 
	WHERE brand = 'Levi'
	GROUP BY t_shirt_id) a 
LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
LIMIT 5;
SQLResult: [(Decimal('31656.000000'),)]
Answer:The total revenue for Levi brand t-shirts after applying discounts is $31,656.
> Finished chain.


In [37]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('31656'),)]
Answer:The total revenue generated from Levi brand t-shirts is $31,656.
> Finished chain.


This is right answer.

In [38]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT COUNT(*) AS count FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(4,)]
Answer:There are 4 white color Levi's t shirts available.
> Finished chain.


This is wrong answer.

In [39]:
qns5 = db_chain.run("SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' LIMIT 5
SQLResult: [(Decimal('251'),)]
Answer:251
> Finished chain.


In [48]:
qns4

'The total revenue generated from Levi brand t-shirts is $31,656.'

## 4. Few Shot Learning

In [49]:
few_shots = [
    
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},

    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},

    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,

     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},

    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     
     }
]

## 5. Making OpenAI Embeddings

In [50]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

e = embeddings.embed_query("How many white color Levi's shirt I have?")
len(e)

C:\Users\shahe\AppData\Local\Temp\ipykernel_26496\3104679062.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


1536

In [53]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query There are currently 40 Nike XS White t-shirts in stock.",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query The price of the inventory for all small size t-shirts is $19,133.",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query The total revenue for Levi brand t-shirts after applying discounts is $31,656

## 6. Querying thorugh Few Shot Learning

In [56]:
from langchain .vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [57]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vectorstore,
    k = 2
)
example_selector.select_examples({'Question': "How many Adidas T shirts I have left in my store?"})

[{'SQLResult': 'Result of the SQL query',
  'Answer': 'There are currently 40 Nike XS White t-shirts in stock.',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'"},
 {'Answer': '251',
  'Question': "How many white color Levi's shirt I have?",
  'SQLResult': 'Result of the SQL query',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"}]

In [ ]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [66]:
    mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use CURDATE() function to get the current date, if the question involves "today".
    
    Use the following format:
    
    Question: Question here
    SQLQuery: Query to run with no pre-amble
    SQLResult: Result of the SQLQuery
    Answer: Final answer here
    
    <query>

    Only use the following tables:
    
    CREATE TABLE discounts (
        discount_id INTEGER NOT NULL AUTO_INCREMENT, 
        t_shirt_id INTEGER NOT NULL, 
        pct_discount DECIMAL(5, 2), 
        PRIMARY KEY (discount_id), 
        CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
        CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
    )DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

    /*
    3 rows from discounts table:
    discount_id	t_shirt_id	pct_discount

    */


    CREATE TABLE t_shirts (
        t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
        brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
        color ENUM('Red','Blue','Black','White') NOT NULL, 
        size ENUM('XS','S','M','L','XL') NOT NULL, 
        price INTEGER, 
        stock_quantity INTEGER NOT NULL, 
        PRIMARY KEY (t_shirt_id), 
        CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
    )DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

    /*
    3 rows from t_shirts table:
    t_shirt_id	brand	color	size	price	stock_quantity
    1	Nike	Blue	XS	43	21
    2	Van Huesen	Blue	XS	34	73
    3	Nike	Blue	L	23	69
    */

    Question: {input}
        No pre-amble.
    """

In [67]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [68]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], # These variables are used in the prefix and suffix
)

In [69]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [70]:
new_chain("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('251'),)]
Answer:There are currently 251 Levi White t-shirts in stock.
> Finished chain.


{'query': "How many white color Levi's t shirts we have available?",
 'result': 'There are currently 251 Levi White t-shirts in stock.'}

In [71]:
new_chain('How much is the price of the inventory for all sized t-shirts?')



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all sized t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts
SQLResult: [(Decimal('106837'),)]
Answer:The total price of the inventory for all sized t-shirts is $106,837.
> Finished chain.


{'query': 'How much is the price of the inventory for all sized t-shirts?',
 'result': 'The total price of the inventory for all sized t-shirts is $106,837.'}

In [72]:
new_chain('How much is the price of the inventory for all large sized t-shirts?')



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all large sized t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'L'
SQLResult: [(Decimal('23150'),)]
Answer:The price of the inventory for all large size t-shirts is $23,150.
> Finished chain.


{'query': 'How much is the price of the inventory for all large sized t-shirts?',
 'result': 'The price of the inventory for all large size t-shirts is $23,150.'}

In [73]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * stock_quantity * (1 - (pct_discount/100))) AS total_revenue FROM t_shirts INNER JOIN discounts ON t_shirts.t_shirt_id=discounts.t_shirt_id WHERE brand='Nike'
SQLResult: [(None,)]
Answer:The total revenue for Nike brand t-shirts after applying discounts is $2,544.
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Nike brand t-shirts after applying discounts is $2,544.'}

With the use of few shot learning, the model tends to learn and provide better answer to the query.